In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import dash_daq as daq
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import datetime

In [102]:
df = pd.read_csv('./matomo_new.csv')  # 讀取資料
df['訪問日期'] = pd.to_datetime(df['訪問日期'])  # 轉換日期格式
df_yesterday = df.query("訪問日期 == '%s'" % df['訪問日期'].max())


,訪問日期,訪問網站,使用裝置,是否為會員,訪問來源,訪問人次,訪問用戶,hours_0,hours_1,hours_2,...,社區收藏,快售服務,實價登錄,追蹤實價登錄,好房新聞瀏覽,好房新聞追蹤,好房TV瀏覽,好房TV追蹤,站內廣告曝光,站內廣告點擊
0,2020-02-01,好房網,app,非會員,直接進入,141327,10205,7409,6751,4775,...,1,0,0,0,114,0,6,0,19972,391
1,2020-02-01,好房網,app,會員,直接進入,28226,1202,1415,1218,889,...,3,0,0,0,26,0,14,5,1862,45


In [50]:
# YC/HF各種統計數字
# (visits)最新一天流量(總流量/會員流量/會員人數/非會員流量)
visits = {
    'YC': {
        'all': df_yesterday[df_yesterday['訪問網站'] == '房仲網']['訪問人次'].sum(),
        'member': df_yesterday[df_yesterday['訪問網站'] == '房仲網'][df_yesterday['是否為會員'] == '會員']['訪問人次'].sum(),
        'other': df_yesterday[df_yesterday['訪問網站'] == '房仲網'][df_yesterday['是否為會員'] == '非會員']['訪問人次'].sum(),
        'member_unique': df_yesterday[df_yesterday['訪問網站'] == '房仲網'][df_yesterday['是否為會員'] == '會員']['訪問用戶'].sum(),
    }, 
 'HF': {
     'all': df_yesterday[df_yesterday['訪問網站'] == '好房網']['訪問人次'].sum(),     
     'member': df_yesterday[df_yesterday['訪問網站'] == '好房網'][df_yesterday['是否為會員'] == '會員']['訪問人次'].sum(),
     'other': df_yesterday[df_yesterday['訪問網站'] == '好房網'][df_yesterday['是否為會員'] == '非會員']['訪問人次'].sum(),
     'member_unique': df_yesterday[df_yesterday['訪問網站'] == '好房網'][df_yesterday['是否為會員'] == '會員']['訪問用戶'].sum(),
    }
}
visits

C:\Users\g6424\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\g6424\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\g6424\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\g6424\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\g6424\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\g6424\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning:

Boolean Series key will be reindexed to m

{'YC': {'all': 1395529,
  'member': 263032,
  'other': 1132497,
  'member_unique': 12648},
 'HF': {'all': 805362,
  'member': 56101,
  'other': 749261,
  'member_unique': 4852}}

In [ ]:
# 前30天/前一年折線圖-YC
days = 30
df_timeSeries = df.query("訪問日期 >= '%s'" % (df['訪問日期'].max() - datetime.timedelta(days=days)))
df_timeSeries_YC = df_timeSeries[df_timeSeries['訪問網站'] == '房仲網'].groupby(['訪問日期', '使用裝置']).sum().reset_index().iloc[:, [0,1,2]]
df_timeSeries_YC_app = df_timeSeries_YC[df_timeSeries_YC['使用裝置'] == 'app']
df_timeSeries_YC_MobilePhone = df_timeSeries_YC[df_timeSeries_YC['使用裝置'] == 'MobilePhone']
df_timeSeries_YC_Desktop = df_timeSeries_YC[df_timeSeries_YC['使用裝置'] == 'Desktop']
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_timeSeries_YC_app['訪問日期'], y=df_timeSeries_YC_app['訪問人次'],
                    mode='lines+markers',
                    name='app'))
fig.add_trace(go.Scatter(x=df_timeSeries_YC_MobilePhone['訪問日期'], y=df_timeSeries_YC_MobilePhone['訪問人次'],
                    mode='lines+markers',
                    name='MobilePhone'))
fig.add_trace(go.Scatter(x=df_timeSeries_YC_Desktop['訪問日期'], y=df_timeSeries_YC_Desktop['訪問人次'],
                    mode='lines+markers',
                    name='Desktop'))
fig.show()

In [ ]:
# 前30天/前一年折線圖-HF
days = 30
df_timeSeries = df.query("訪問日期 >= '%s'" % (df['訪問日期'].max() - datetime.timedelta(days=days)))
df_timeSeries_HF = df_timeSeries[df_timeSeries['訪問網站'] == '好房網'].groupby(['訪問日期', '使用裝置']).sum().reset_index().iloc[:, [0,1,2]]
df_timeSeries_HF_app = df_timeSeries_HF[df_timeSeries_HF['使用裝置'] == 'app']
df_timeSeries_HF_MobilePhone = df_timeSeries_HF[df_timeSeries_HF['使用裝置'] == 'MobilePhone']
df_timeSeries_HF_Desktop = df_timeSeries_HF[df_timeSeries_HF['使用裝置'] == 'Desktop']
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_timeSeries_HF_app['訪問日期'], y=df_timeSeries_HF_app['訪問人次'],
                    mode='lines+markers',
                    name='app'))
fig.add_trace(go.Scatter(x=df_timeSeries_HF_MobilePhone['訪問日期'], y=df_timeSeries_HF_MobilePhone['訪問人次'],
                    mode='lines+markers',
                    name='MobilePhone'))
fig.add_trace(go.Scatter(x=df_timeSeries_HF_Desktop['訪問日期'], y=df_timeSeries_HF_Desktop['訪問人次'],
                    mode='lines+markers',
                    name='Desktop'))
fig.show()

In [120]:
# YC每小時各裝置流量
df_yesterday_time_YC = df_yesterday[df_yesterday['訪問網站'] == '房仲網'].groupby('使用裝置').sum()
df_yesterday_time_YC = df_yesterday_time_YC.iloc[:, [i for i in range(2, 26)]].reset_index().melt(id_vars=["使用裝置"], 
                                                          var_name="時段", 
                                                          value_name="訪問人次")
fig = px.bar(df_yesterday_time_YC, x="時段", y="訪問人次", color='使用裝置', barmode='group')
fig.show()

,使用裝置,時段,訪問人次
0,Desktop,hours_0,10780
1,MobilePhone,hours_0,28244
2,app,hours_0,29835
3,Desktop,hours_1,4733
4,MobilePhone,hours_1,17013
...,...,...,...
67,MobilePhone,hours_22,42792
68,app,hours_22,40429
69,Desktop,hours_23,9637
70,MobilePhone,hours_23,31684


In [ ]:
# HF每小時各裝置流量
df_yesterday_time_HF = df_yesterday[df_yesterday['訪問網站'] == '好房網'].groupby('使用裝置').sum()
df_yesterday_time_HF = df_yesterday_time_HF.iloc[:, [i for i in range(2, 26)]].reset_index().melt(id_vars=["使用裝置"], 
                                                          var_name="時段", 
                                                          value_name="訪問人次")
fig = px.bar(df_yesterday_time_HF, x="時段", y="訪問人次", color='使用裝置', barmode='group')
fig.show()

In [ ]:
# 訪問來源烈日圖
themeColor =  px.colors.diverging.RdYlBu
df_yesterday_referer = df_yesterday.iloc[:, [1,4,5]].groupby(['訪問網站', '訪問來源']).sum().reset_index()
fig = px.sunburst(df_yesterday_referer, 
                      path=['訪問網站', '訪問來源'],  # 分類變數
                      values='訪問人次',  # 面積依據資料
                      color='訪問網站',  # 分色變數
                      hover_data=['訪問網站'],  # 游標顯示資料
                     )
fig.update_traces(
     marker=dict(colors=themeColor, line=dict(color='#ffffff', width=2))
)
fig.update_layout(
        margin=dict(l=10, r=10, t=10, b=10),  # 圖形在畫布的邊界設定
        paper_bgcolor="rgba(0,0,0,0)",   # 畫布背景顏色
        plot_bgcolor="rgba(0,0,0,0)",  # 圖形背景顏色
        autosize=True,  # 自動調整大小
        font_color='#000000',  # 文字顏色
        font_size=20,  # 文字大小
        font_family='Microsoft JhengHei',  # 字體
    )
fig.show()

In [133]:
# 廣告曝光/點擊效益
df_yesterday_ad = df_yesterday.iloc[:, [1, -2,-1]].groupby('訪問網站').sum().reset_index().melt(id_vars=["訪問網站"], 
                                                          var_name="行為", 
                                                          value_name="人次")
fig = px.sunburst(df_yesterday_ad, 
                      path=['訪問網站', '行為'],  # 分類變數
                      values='人次',  # 面積依據資料
                      color='訪問網站',  # 分色變數
                      hover_data=['訪問網站'],  # 游標顯示資料
                     )
fig.update_traces(
     marker=dict(line=dict(color='#ffffff', width=2))
)
fig.update_layout(
        margin=dict(l=10, r=10, t=10, b=10),  # 圖形在畫布的邊界設定
        paper_bgcolor="rgba(0,0,0,0)",   # 畫布背景顏色
        plot_bgcolor="rgba(0,0,0,0)",  # 圖形背景顏色
        autosize=True,  # 自動調整大小
        font_color='#000000',  # 文字顏色
        font_size=20,  # 文字大小
        font_family='Microsoft JhengHei',  # 字體
    )
fig.show()

,訪問網站,行為,人次
0,好房網,站內廣告點擊,8481
1,房仲網,站內廣告點擊,14419
2,好房網,站內廣告曝光,279495
3,房仲網,站內廣告曝光,495052


In [ ]:
# 網站服務使用排行
# YC
df_yesterday_function_YC = df_yesterday[df_yesterday['訪問網站'] == '房仲網'].iloc[:, [1,2,33,35,37,38,40,41]].groupby(['訪問網站', '使用裝置']).sum().reset_index()
df_yesterday_function_YC = df_yesterday_function_YC.melt(id_vars=['訪問網站', '使用裝置'], var_name="功能", value_name="人次")
df_yesterday_function_YC = df_yesterday_function_YC.sort_values(by=['人次','訪問網站', '使用裝置', '功能'], ascending=True).reset_index(drop=True)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=df_yesterday_function_YC['功能'].unique(),
    x=df_yesterday_function_YC[df_yesterday_function_YC['使用裝置'] == 'app']['人次'],
    name='app',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=df_yesterday_function_YC['功能'].unique(),
    x=df_yesterday_function_YC[df_yesterday_function_YC['使用裝置'] == 'MobilePhone']['人次'],
    name='MobilePhone',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=df_yesterday_function_YC['功能'].unique(),
    x=df_yesterday_function_YC[df_yesterday_function_YC['使用裝置'] == 'Desktop']['人次'],
    name='Desktop',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))

fig.update_layout(barmode='stack')
fig.show()

In [ ]:
# 網站服務使用排行
# HF
df_yesterday_function_HF = df_yesterday[df_yesterday['訪問網站'] == '好房網'].iloc[:, [1,2,33,35,37,38,41,43,45]].groupby(['訪問網站', '使用裝置']).sum().reset_index()
df_yesterday_function_HF = df_yesterday_function_HF.melt(id_vars=['訪問網站', '使用裝置'], var_name="功能", value_name="人次")
df_yesterday_function_HF = df_yesterday_function_HF.sort_values(by=['人次','訪問網站', '使用裝置', '功能'], ascending=True).reset_index(drop=True)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=df_yesterday_function_HF['功能'].unique(),
    x=df_yesterday_function_HF[df_yesterday_function_HF['使用裝置'] == 'app']['人次'],
    name='app',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=df_yesterday_function_HF['功能'].unique(),
    x=df_yesterday_function_HF[df_yesterday_function_HF['使用裝置'] == 'MobilePhone']['人次'],
    name='MobilePhone',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=df_yesterday_function_HF['功能'].unique(),
    x=df_yesterday_function_HF[df_yesterday_function_HF['使用裝置'] == 'Desktop']['人次'],
    name='Desktop',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))

fig.update_layout(barmode='stack')
fig.show()